<a href="https://colab.research.google.com/github/Omar-Abd-Elwahab/VCF_filter/blob/main/data_preparation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install pyvcf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2 python3-pysam python3-vcf
The following NEW packages will be installed:
  libhts3 libhtscodecs2 python3-pysam python3-vcf pyvcf
0 upgraded, 5 newly installed, 0 to remove and 31 not upgraded.
Need to get 2,243 kB of archives.
After this operation, 7,117 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pysam amd64 0.17.0+ds-2build1 [1,749 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-vcf amd64 0.6.8+git20170215.476169c-8build1 [43.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pyvcf all 0.6.8+git20170215.476169c-8build

In [ ]:
!apt-get install -y tabix
!pip install pytabix


In [ ]:
# Opening the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd ./drive/MyDrive/Colab\ Notebooks/deepref


In [ ]:
pwd

In [ ]:

!grep -o -i PASS HG003_GRCh38_1_22_v4.2.1_benchmark.vcf | wc -l
!grep -o -i FAIL HG003_GRCh38_1_22_v4.2.1_benchmark.vcf | wc -l

In [ ]:
import vcf

#if a variat exist in both a VCF file anf the reference VCF, the variant FILTER is set to PASS. Else, it is FAIL
def update_filter_column(vcf_file, reference_vcf, output_vcf):
    vcf_reader = vcf.Reader(filename=vcf_file)
    reference_reader = vcf.Reader(filename=reference_vcf)

    reference_variants = set((record.POS) for record in reference_reader)

    vcf_writer = vcf.Writer(open(output_vcf, 'w'), vcf_reader)

    for record in vcf_reader:
        variant_key = (record.POS)
        if variant_key in reference_variants:
            record.FILTER = "PASS"

        else:
            record.FILTER = "FAIL"

        vcf_writer.write_record(record)





In [ ]:
# Provide the paths to your VCF files
#file_1 = "HG003_platypus.vcf"
file_2 = "HG003-bcf.vcf"
file_4 = "HG003_gatk.vcf"
reference_vcf_path = "HG003_GRCh38_1_22_v4.2.1_benchmark.vcf"
#output_1 = "/content/HG003_platypus_updated.vcf"
output_2 = "/content/HG003_bcf_updated.vcf"
output_4 = "/content/HG003_gatk_updated.vcf"

#update_filter_column(file_1, reference_vcf_path, output_1)
update_filter_column(file_2, reference_vcf_path, output_2)
update_filter_column(file_4, reference_vcf_path, output_4)


In [ ]:
cd /content

In [ ]:
!apt-get install vcftools


In [ ]:
pwd

In [ ]:
ls

In [ ]:
#print the number of fail variants
#!grep -o -i FAIL HG003_platypus_updated.vcf | wc -l
!grep -o -i FAIL HG003_bcf_updated.vcf | wc -l
!grep -o -i FAIL HG003_gatk_updated.vcf | wc -l

#!grep -o -i PASS HG003_platypus_updated.vcf | wc -l
!grep -o -i PASS HG003_bcf_updated.vcf | wc -l
!grep -o -i PASS HG003_gatk_updated.vcf | wc -l








In [ ]:
!ls -lh

In [ ]:
!vcf-concat HG003_bcf_updated.vcf \
  HG003_gatk_updated.vcf > HG003_merged.vcf

In [ ]:
ls -lh

In [ ]:
!cat HG003_merged.vcf | vcf-annotate --fill-type | grep -oP "TYPE=\w+" | sort | uniq -c

In [ ]:
!grep -o -i PASS HG003_merged.vcf | wc -l
!grep -o -i FAIL HG003_merged.vcf | wc -l

In [ ]:
#!vcftools --vcf HG003_merged.vcf --remove-indels --recode --recode-INFO-all --out SNPs_merged_hg003

In [ ]:
#!cat SNPs_merged_hg003.recode.vcf | vcf-annotate --fill-type | grep -oP "TYPE=\w+" | sort | uniq -c

In [ ]:
#!vcftools --vcf HG003_merged.vcf --keep-only-indels --recode --recode-INFO-all --out INDELs_merged_hg003

In [ ]:
#!cat INDELs_merged_hg003.recode.vcf | vcf-annotate --fill-type | grep -oP "TYPE=\w+" | sort | uniq -c

In [ ]:
#find unique words to be added later in the vocab list for the TRNASFORMERS model
def find_words_between_substring(file_path):
    found_words = []

    with open(file_path, 'r') as file:
        content = file.read()
        start_index = content.find("ID=")

        while start_index != -1:
            start_index += 3  # Length of "ID="
            end_index = content.find(",", start_index)
            if end_index != -1:
                found_words.append(content[start_index:end_index].strip())
            start_index = content.find("ID=", end_index)

    return found_words




In [ ]:
# Usage example
file_path_1 = "HG003_bcf_updated.vcf"  # Replace with your file path
#file_path_2 = "HG003_platypus_updated.vcf"
file_path_3 = "HG003_gatk_updated.vcf"
result_1 = find_words_between_substring(file_path_1)
print(result_1)
#result_2 = find_words_between_substring(file_path_2)
#print(result_2)
result_3 = find_words_between_substring(file_path_3)
print(result_3)



In [ ]:
#remove not needed words
print(result_1.index('chr1'))
print(result_3.index('chr1'))


In [ ]:
result_1 = result_1[:21]
result_3 = result_3[1:22]

In [ ]:
out_1 = map(lambda x:x.lower(), result_1)
#out_2 = map(lambda x:x.lower(), result_2)
out_3 = map(lambda x:x.lower(), result_3)


result_1 = list(out_1)
#result_2 = list(out_2)
result_3 = list(out_3)

In [ ]:

print(result_1)
#print(result_2)
print(result_3)

In [ ]:
!head bert-base-uncased-vocab.txt

In [ ]:
!wc -l bert-base-uncased-vocab.txt

In [ ]:
#add unique words to the bert_base_uncased vocabs
with open("bert-base-uncased-vocab.txt", "a") as f:
   for word in result_1:
     f.write(word+'\n')
   for z in result_3:
     f.write(z+'\n')




In [ ]:
!wc -l bert-base-uncased-vocab.txt

In [ ]:
def remove_duplicate_words(file_path, output_file_path):
    unique_words = []
    seen_words = set()

    with open(file_path, 'r') as file:
        for line in file:
            words = line.strip().split()
            for word in words:
                if word not in seen_words:
                    unique_words.append(word)
                    seen_words.add(word)

    with open(output_file_path, 'w') as output_file:
        for word in unique_words:
            output_file.write(word + '\n')
# Usage example
input_file_path = "bert-base-uncased-vocab.txt"  # Replace with your input file path
output_file_path = "unique_word.txt"  # Replace with your desired output file path
remove_duplicate_words(input_file_path, output_file_path)


In [ ]:
!wc -l unique_word.txt


In [ ]:
!tail -26 unique_word.txt

In [ ]:
#!awk '/^##/{h=$0; next} /^#CHROM/{print h; print; next} {print}' SNPs_merged_hg003.recode.vcf > hg003_snps.vcf
#!awk '/^##/{h=$0; next} /^#CHROM/{print h; print; next} {print}' INDELs_merged_hg003.recode.vcf > hg003_indel.vcf
!awk '/^##/{h=$0; next} /^#CHROM/{print h; print; next} {print}' HG003_merged.vcf > hg003_merged.vcf



awk: cannot open HG003_merged.vcf (No such file or directory)


In [ ]:
!head hg003_merged.vcf

In [ ]:
#!sed -i '1d' hg003_snps.vcf
#!sed -i '1d' hg003_indel.vcf
!sed -i '1d' hg003_merged.vcf


In [ ]:
!head -15 hg003_merged.vcf

In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("hg003_merged.vcf", delimiter='\t', header=None, names=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'HG003'], low_memory=False)


df=df.drop(df.columns[[0, 1, 2, 3, 4]], axis=1)


# Report the number of sentences.
print('Number of training lines: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
print(df.sample(3, random_state=42))


In [ ]:
df['FILTER'].replace(['PASS', 'FAIL'],
                        [0, 1], inplace=True)


# Display 10 random rows from the data.
df.sample(10, random_state=42)

In [ ]:
df.to_csv('hg003_bcf_gatk.csv', index=False)


In [ ]:
!cp hg003_bcf_gatk.csv ./drive/MyDrive/Colab\ Notebooks/deepref



In [ ]:
!cp unique_word.txt ./drive/MyDrive/Colab\ Notebooks/deepref


In [ ]:
!wc -l unique_word.txt
!wc -l bert-base-uncased-vocab.txt

In [ ]:
ls -lh ./drive/MyDrive/Colab\ Notebooks/deepref